In [2]:
import numpy as np
from time import time
from functools import wraps

#### Timing Wrapper Function

We will use this wrapper to see how long it takes to execute each function

In [3]:
def timing(f):
    @wraps(f)
    def wrap(*args, **kw):
        ts = time()
        result = f(*args, **kw)
        te = time()
        print(f'func:{f.__name__} took: {te-ts:.4f} sec')
        return result
    return wrap

#### Binomial Tree Representation

A stock price tree can be represented using nodes $(i, j)$, with an initial stock price $S_0$. The stock price at any given node is defined by:

$$S_{i,j} = S_0 \, u^j d^{i-j}$$

Where:
* $i$ is the time step.
* $j$ is the number of upward movements.
* $C_{i,j}$ denotes the option (contract) value at node $(i, j)$.

At maturity ($i = N$), $C_{N,j}$ represents the terminal payoff.



#### American Option Characteristics

Consider an **American put option**. At maturity $T = t_N$, the payoff at the terminal nodes is:

$$C_{N,j} = \max(K - S_{N,j}, 0)$$

For all earlier nodes $(i, j)$, the option value is the maximum of the **exercise value** (intrinsic value) and the **continuation value** (discounted expected future value):

$$C_{i,j} = \max \left( K - S_{i,j}, e^{-r \Delta t} [q C_{i+1, j+1} + (1 - q) C_{i+1, j}] \right)$$

In [21]:
S0 = 100 # Initial stock price
K = 100 # Strike price
T = 1 # Time to maturity in years
r = 0.06 # Risk-free interest rate
N = 3 # Number of time steps
u = 1.1 # Up factor
d = 1/u # Down factor
opttype = 'P' # 'C' for Call, 'P' for Put

#### American Tree Slow

In [33]:
import numpy as np


@timing
def american_slow_tree(K, T, S0, r, N, u, d, optype="P"):
    optype = optype.strip().upper()

    dt = T / N
    q = (np.exp(r * dt) - d) / (u - d)
    disc = np.exp(-r * dt)

    # stock prices at maturity (i = N)
    S = np.zeros(N + 1)
    for j in range(N + 1):
        S[j] = S0 * (u**j) * (d**(N - j))

    # option payoff at maturity
    C = np.zeros(N + 1)
    for j in range(N + 1):
        if optype == "P":
            C[j] = max(0.0, K - S[j])
        else:
            C[j] = max(0.0, S[j] - K)

    # backward recursion
    for i in range(N - 1, -1, -1):
        for j in range(i + 1):
            S_ij = S0 * (u**j) * (d**(i - j))          # scalar at node (i,j)
            hold = disc * (q * C[j + 1] + (1 - q) * C[j])

            if optype == "P":
                exercise = K - S_ij
            else:
                exercise = S_ij - K

            C[j] = max(hold, exercise)

    return C[0]


In [28]:
american_slow_tree_price = american_slow_tree(K, T, S0, r, N, u, d, optype="P")
print(american_slow_tree_price)

4.654588754602527


#### American Tree Fast

In [34]:
import numpy as np

@timing
def american_fast_tree(K, T, S0, r, N, u, d, optype="P"):
    # Precompute Values
    optype = optype.strip().upper()
    dt = T / N
    q = (np.exp(r * dt) - d) / (u - d)
    disc = np.exp(-r * dt)

    # stock prices at maturity (i = N)
    S = S0 * d**(np.arange(N, -1, -1)) * u**(np.arange(0, N + 1))


    # option payoff at maturity
    if optype == "P":
        C = np.maximum(0.0, K - S)
    else:
        C = np.maximum(0.0, S - K)

    # backward recursion
    for i in range(N - 1, -1, -1):
        S = S0 * d**(np.arange(i, -1, -1)) * u**(np.arange(0, i + 1))
        C[:i +1] = disc * (q *C[1:i + 2] + (1 - q) * C[0:i + 1])
        C = C[:-1]

        if optype == "P":
            exercise = K - S
        else:
            exercise = S - K
        
        C = np.maximum(C, exercise)
    

    return C[0]

american_fast_tree_price = american_fast_tree(K, T, S0, r, N, u, d, optype="P")
print(american_fast_tree_price)

func:american_fast_tree took: 0.4625 sec
98.13190292991497


#### Comparison in Code (Slow vs Fast)

In [35]:
for N in [3, 50 , 100, 1000, 5000]:
    american_fast_tree_price = american_fast_tree(K, T, S0, r, N, u, d, optype="P")
    american_slow_tree_price = american_slow_tree(K, T, S0, r, N, u, d, optype="P")
    print(f"N={N}: Fast Price={american_fast_tree_price}, Slow Price={american_slow_tree_price}")

func:american_fast_tree took: 0.0000 sec
func:american_slow_tree took: 0.0000 sec
N=3: Fast Price=4.654588754602527, Slow Price=4.654588754602527
func:american_fast_tree took: 0.0010 sec
func:american_slow_tree took: 0.0010 sec
N=50: Fast Price=23.423684208556715, Slow Price=23.423684208556715
func:american_fast_tree took: 0.0010 sec
func:american_slow_tree took: 0.0051 sec
N=100: Fast Price=33.43686702508116, Slow Price=33.43686702508116
func:american_fast_tree took: 0.0276 sec
func:american_slow_tree took: 0.4325 sec
N=1000: Fast Price=83.184163941043, Slow Price=83.184163941043
func:american_fast_tree took: 0.4324 sec
func:american_slow_tree took: 10.7935 sec
N=5000: Fast Price=98.13190292991497, Slow Price=98.13190292991497
